In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('./20210605_074859_39194_wkvzn.csv')

In [4]:
#data.info()

In [5]:
data.head()

,#user_id,#account_id,新增日期,激励pv,激励ecpm,其他pv,其他ecpm,关卡数,cpa,7日回收
0,845694607653744642,03ab6210ad8121bc,2021-05-22,8,80.31,14,5.36,1,3.50,5.97
1,845699201877938176,48c17aa6f6e74d57,2021-05-22,8,111.63,22,17.73,3,3.50,1.28
2,845622218721206272,6476ecd7bf317f44,2021-05-22,32,37.42,164,6.51,16,3.26,4.05
3,845382590654582784,6c9031f696befaf9,2021-05-21,13,81.88,32,7.44,5,3.62,1.30
4,845356688825004034,6ef6ed5640266318,2021-05-21,46,35.44,113,5.88,16,3.62,2.29


In [6]:
data['回收率'] = data['7日回收']/data['cpa']

In [7]:
data['是否回收'] = data['回收率'].apply(lambda x: 1 if x>=1 else 0)

In [8]:
data.head()

,#user_id,#account_id,新增日期,激励pv,激励ecpm,其他pv,其他ecpm,关卡数,cpa,7日回收,回收率,是否回收
0,845694607653744642,03ab6210ad8121bc,2021-05-22,8,80.31,14,5.36,1,3.50,5.97,1.705714,1
1,845699201877938176,48c17aa6f6e74d57,2021-05-22,8,111.63,22,17.73,3,3.50,1.28,0.365714,0
2,845622218721206272,6476ecd7bf317f44,2021-05-22,32,37.42,164,6.51,16,3.26,4.05,1.242331,1
3,845382590654582784,6c9031f696befaf9,2021-05-21,13,81.88,32,7.44,5,3.62,1.30,0.359116,0
4,845356688825004034,6ef6ed5640266318,2021-05-21,46,35.44,113,5.88,16,3.62,2.29,0.632597,0


In [9]:
data['是否回收'].value_counts()

0    1849
1     946
Name: 是否回收, dtype: int64

In [10]:
data['#user_id'].nunique()

2795

In [11]:
X = data[['激励pv', '激励ecpm', '其他pv', '其他ecpm',
       '关卡数']]

In [12]:
y = data['是否回收']

In [13]:
X.head()

,激励pv,激励ecpm,其他pv,其他ecpm,关卡数
0,8,80.31,14,5.36,1
1,8,111.63,22,17.73,3
2,32,37.42,164,6.51,16
3,13,81.88,32,7.44,5
4,46,35.44,113,5.88,16


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
#sv.score(X_test, y_test)

In [18]:
#rfc.score(X_test, y_test)

In [32]:
import json
import os
import time
import uuid

import boto3
import sagemaker

region = boto3.Session().region_name
container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
hyperparameters = {
    "max_depth":"5",
    "objective": "binary:logistic",
    "num_round": "4",
    "eval_metric": "auc"
}
# train_content_type = (
#     "application/x-parquet" if output_content_type.upper() == "PARQUET"
#     else "text/csv"
# )
train_input = sagemaker.inputs.TrainingInput(
    s3_data=f"s3://sagemaker-studio-g8hgbwmv6h6/data_new5.csv",
    content_type="text/csv",
)

In [33]:
iam_role = sagemaker.get_execution_role()

estimator = sagemaker.estimator.Estimator(
    container,
    iam_role,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
)
estimator.fit({"train": train_input})

2021-06-08 10:30:51 Starting - Starting the training job...
2021-06-08 10:30:54 Starting - Launching requested ML instancesProfilerReport-1623148251: InProgress
......
2021-06-08 10:32:18 Starting - Preparing the instances for training......
2021-06-08 10:33:18 Downloading - Downloading input data...
2021-06-08 10:33:48 Training - Training image download completed. Training in progress.
2021-06-08 10:33:48 Uploading - Uploading generated training model.[2021-06-08 10:33:44.654 ip-10-0-78-255.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.traini

In [ ]:
# import xgboost as xgb

In [ ]:
# sgbclf = xgb.XGBClassifier().fit(X_train, y_train)

In [ ]:
# sgbclf.score(X_test, y_test)

mlp = MLPClassifier(hidden_layer_sizes=(50, 100), max_iter=100).fit(X_train,y_train)

mlp.score(X_train, y_train)

mlp.score(X_test, y_test)

from sklearn.metrics import confusion_matrix



In [ ]:
from sagemaker.serializers import CSVSerializer
xgb_predictor = estimator.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.m4.xlarge',
    serializer=CSVSerializer())

In [ ]:
response = xgb_predictor.predict(input).decode('utf-8')
print(response)